##Step 0: Persist everything to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')          # Skip this cell if you don't need persistence

# Choose a root dir that survives VM resets if you mounted Drive
ROOT = "/content/drive/MyDrive/ransomware_lab"  # or "/content" if Drive not mounted
!mkdir -p $ROOT

##Step 1: Install Tools

In [ ]:
%%bash
sudo apt-get update -qq
sudo apt-get install -y restic gnupg tree

In [ ]:
#Install extra utilities
%%bash
sudo apt-get install -y inotify-tools   # for near real‑time backup
pip install --quiet watchdog            # Python file‑watcher library

##Step 2: Configure Restic Repo

In [ ]:
import os, subprocess, textwrap, getpass, json, pathlib, shutil, time

# Change these if you like
DATA_DIR      = f"{ROOT}/victim_data"
RESTIC_REPO   = f"{ROOT}/backup_repo"
RESTORE_DIR   = f"{ROOT}/restore"
os.makedirs(DATA_DIR,   exist_ok=True)
os.makedirs(RESTORE_DIR, exist_ok=True)

# One‑liner to set the repo password (DON'T lose it!)
os.environ["RESTIC_PASSWORD"] = "colab‑demo‑super‑secret"  # choose a strong one IRL

# Initialize repository once
if not pathlib.Path(RESTIC_REPO, "config").exists():
    !restic -r $RESTIC_REPO init

##Step 3: Generate sample "production" files

In [ ]:
%%bash -s "$DATA_DIR"
TARGET=$1
mkdir -p "$TARGET/docs" "$TARGET/images"

# simple text data
echo "Quarterly revenue: \$123,456"  > "$TARGET/docs/report_Q1.txt"
echo "User list (PII redacted)"      > "$TARGET/docs/users.txt"

# simulate binary data
head -c 1M </dev/urandom > "$TARGET/images/raw_sensor_dump.bin"

tree -h "$TARGET"

##Step 4: Start Watchdog that Snapshots on Every File-close

In [ ]:
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import subprocess, os

class ResticOnClose(FileSystemEventHandler):
    def __init__(self, repo, path): self.repo, self.path = repo, path
    def on_closed(self, event):
        if event.is_directory: return
        print(f"[Watch] {event.src_path} closed → instant backup")
        subprocess.run(
            ["restic","-r",self.repo,"backup",self.path,"--tag","auto"],
            env=os.environ, check=False
        )

observer = Observer()
observer.schedule(ResticOnClose(RESTIC_REPO, DATA_DIR), str(DATA_DIR), recursive=True)
observer.start()
print("📡 Watchdog active — every write triggers a restic backup")

##Step 5: Start Background Producer
Writes new log file every 15 seconds

In [ ]:
import threading, random, string, pathlib, time
STREAM_DIR = pathlib.Path(DATA_DIR) / "logs"; STREAM_DIR.mkdir(exist_ok=True)

def producer(stop_event, interval=15):
    while not stop_event.is_set():
        fn = STREAM_DIR / f"log_{int(time.time())}.txt"
        fn.write_text(''.join(random.choices(string.ascii_letters, k=1024)))
        print(f"[Producer] wrote {fn.name}")
        time.sleep(interval)

stop_event = threading.Event()
prod_thr   = threading.Thread(target=producer, args=(stop_event,), daemon=True)
prod_thr.start()

##Step 6: First Backup ("golden")
Initial Baseline snapshot

In [ ]:
%%bash -s "$DATA_DIR" "$RESTIC_REPO"
DATA=$1; REPO=$2
restic -r "$REPO" backup "$DATA" --tag "baseline"
restic -r "$REPO" snapshots

##Step 7: 15 Seconds Pause
Producer and Watchdog can create auto snapshots

In [ ]:
import time, subprocess, os
print("⌛ Sleeping 15 s so real‑time activity accumulates …")
time.sleep(15)
!restic -r $RESTIC_REPO snapshots | tail -n +3       # show newest snapshots

##Step 8: Simulate Ransomware attack

In [ ]:
%%bash -s "$DATA_DIR"
VICTIM=$1
# Encrypt *.txt as a stand‑in for ransomware, then delete originals
cd "$VICTIM/docs"
for f in *.txt; do
  openssl enc -aes-256-cbc -md sha256 -salt -in "$f" -out "${f}.enc" -pass pass:evil123
  shred -u "$f"
done
echo "After attack:"
tree -h "$VICTIM"

##Step 9: Attempt to Read
Will fail

In [ ]:
import pathlib, sys

plaintext = pathlib.Path(DATA_DIR) / "docs" / "report_Q1.txt"   # original file path

try:
    text = plaintext.read_text()          # should raise FileNotFoundError
    print("❌  UNEXPECTED: plaintext still readable!", text[:100])
    sys.exit(1)
except (FileNotFoundError, PermissionError):
    print("✅  Expected failure: plaintext is gone or unreadable — ransomware succeeded.")

##Step 10: Restore the Latest Clean Snapshot

In [ ]:
#List all snapshots
%%bash -s "$RESTIC_REPO"
echo "All snapshots in repo:"
restic -r "$1" snapshots

In [ ]:
#Restore the latest baseline (clean) snapshot
%%bash -s "$RESTIC_REPO" "$RESTORE_DIR"
restic -r "$1" restore latest --tag baseline --target "$2"

echo "✅  Restored baseline snapshot to $2"
tree -h "$2" | head -n 20

##Step 11: Verify Restored File Exists and Matches Expected Content

In [ ]:
%%bash
# 1. Point to your restore directory
export RESTORE_DIR="/content/drive/MyDrive/ransomware_lab/restore"

# 2. Locate the nested victim_data folder
VICTIM_PATH=$(find "$RESTORE_DIR" -type d -name victim_data | head -n1)
if [ -z "$VICTIM_PATH" ]; then
  echo "❌ Could not find victim_data under $RESTORE_DIR"
  exit 1
fi

echo "✅ Restored data is at: $VICTIM_PATH"

# 3. List the docs you recovered
echo; echo "Recovered docs:"
tree -h "$VICTIM_PATH/docs" | sed -n '1,5p'

# 4. Show the contents of report_Q1.txt
echo; echo "Contents of report_Q1.txt:"
cat "$VICTIM_PATH/docs/report_Q1.txt"

In [ ]:
import pathlib, hashlib, sys

# Adjust this to your actual restore path variable if needed
restore_root = pathlib.Path(RESTORE_DIR)
victim = next(restore_root.rglob("victim_data"), None)
if victim is None:
    print("❌  victim_data not found!"); sys.exit(1)

file = victim / "docs" / "report_Q1.txt"
if not file.exists():
    print("❌  report_Q1.txt missing!"); sys.exit(1)

content = file.read_text()
assert "Quarterly revenue: $123,456" in content, "Content mismatch!"

sha = hashlib.sha256(file.read_bytes()).hexdigest()
print(f"✅  report_Q1.txt verified — SHA‑256: {sha[:12]}…")

##Step 12: Compare Original to Restored
Sanity Check

In [ ]:
%%bash -s "$DATA_DIR" "$RESTORE_DIR"
# $1 is the live data dir, $2 is where Restic put your restore tree
ORIG="$1"
RESTORE_ROOT="$2"

# 2. Locate the restored victim_data under RESTORE_ROOT
RESTORED_V=$(find "$RESTORE_ROOT" -type d -name victim_data | head -n1)
if [ -z "$RESTORED_V" ]; then
  echo "❌  Could not find victim_data under $RESTORE_ROOT"
  exit 1
fi

echo "✅  Restored data is at: $RESTORED_V"
echo

# 3. Compare original vs restored
echo "Comparing live vs restored:"
echo "  live:     $ORIG"
echo "  restored: $RESTORED_V"
echo

diff -rq "$ORIG" "$RESTORED_V" \
  && echo "✅  No unexpected differences." \
  || echo "🔶  Differences above are expected for encrypted docs."

##Step 13: Stop Background Producer and Watchdog

In [ ]:
stop_event.set()               # stop producer loop
observer.stop(); observer.join()
print("✅ Real‑time components shut down")

##Optional: Prune Workspace
* Restores only the baseline snapshot (--tag baseline latest).
* Moves the restored victim_data into place, overwriting the encrypted version.
* In victim_data/logs, keeps just the newest .txt file.
* Cleans up temporary files and then prints the top of your refreshed victim_data

In [ ]:
%%bash -s "$RESTIC_REPO" "$DATA_DIR"
# $1 = your Restic repo, $2 = /…/ransomware_lab/victim_data
REPO="$1"
DATA="$2"
PARENT="${DATA%/*}"
TEMP="$PARENT/temp_restore"

echo "ℹ️  Restoring baseline snapshot into temp folder…"
rm -rf "$TEMP" && mkdir -p "$TEMP"
restic -r "$REPO" restore --tag baseline latest --target "$TEMP"

echo "ℹ️  Locating nested victim_data in temp…"
RESTORED=$(find "$TEMP" -type d -name victim_data | head -n1)
if [ -z "$RESTORED" ]; then
  echo "❌  victim_data not found under $TEMP" && exit 1
fi
echo "✅  Found clean snapshot at: $RESTORED"

echo "ℹ️  Replacing live data…"
rm -rf "$DATA"
mv "$RESTORED" "$DATA"

echo "ℹ️  Pruning logs to keep only the latest…"
LOGS="$DATA/logs"
if [ -d "$LOGS" ]; then
  cd "$LOGS"
  LATEST=$(ls -1t *.txt | head -n1)
  echo "   Keeping only: $LATEST"
  ls *.txt | grep -v "^$LATEST\$" | xargs -r rm --
fi

echo "ℹ️  Cleaning up temp folder…"
rm -rf "$TEMP"

echo; echo "🎯 Final victim_data structure:"
tree -h "$DATA" | sed -n '1,10p'
